## ColBERT
#### index etmek cok uzun sürüyor
#### ve retriever üzerinden Fusion calistirdigimda mesela cok uzun sürüyor. Retriever'i kontrol etmem gerekiyor.

In [10]:
question = "Mobil hat faturalarimi nasil görüntüleyebilir ve ödeyebilirim?"

##### Data directory
###### data klasörünün altinda 8 ana kategörü olmali, tam olarak isimler uyusmali

In [11]:
# Define the directory containing the rag data
data_directory = "/Users/taha/Desktop/rag/data"

##### Imports

In [12]:
# Import necessary libraries
import os
import numpy as np
import glob
import gc
import tiktoken
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.summarize import load_summarize_chain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, FewShotChatMessagePromptTemplate, PromptTemplate
from langchain.load import dumps, loads
from langchain.schema import Document
from operator import itemgetter
from typing import Literal, List, Tuple

##### Model

In [13]:
# Load API Keys from environment variables
load_dotenv()  # Load environment variables from a .env file

# Retrieve API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize the chat model and embedding model
# ChatOpenAI is used to interact with the OpenAI GPT model, and OpenAIEmbeddings is used for generating embeddings for documents
model = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [14]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [15]:
import os

# Define the base directory where the subdirectories are located
base_directory = 'data/'

def read_txt_files_exclude_summary(directory):
    """
    Reads the content of all .txt files in subdirectories, excluding _summary.txt files.
    Returns a list of document contents.
    """
    documents = []
    
    # Get all subdirectories in the given directory
    subdirectories = [os.path.join(directory, subdir) for subdir in os.listdir(directory)
                      if os.path.isdir(os.path.join(directory, subdir))]
    
    for subdirectory in subdirectories:
        # List all files in the subdirectory
        for root, _, files in os.walk(subdirectory):
            for file in files:
                # Process only .txt files that are not _summary.txt
                if file.endswith('.txt') and file != '_summary.txt':
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r') as txt_file:
                        content = txt_file.read()
                        documents.append(content)
                        print(f"Read document from '{file_path}'")
    
    return documents

# Function to check if the index already exists
def check_index_exists(index_name, index_root=None):
    """
    Check if an index with the given name exists.
    If an index_root is provided, it will check in that directory.
    """
    if index_root is None:
        index_root = '.ragatouille/'
    
    index_path = os.path.join(index_root, index_name)
    
    # Check if the index path exists
    return os.path.exists(index_path)

# Main logic to either use an existing index or create a new one
index_name = "telekom_rag"
max_document_length = 256
split_documents = True

# First check if the index already exists
if check_index_exists(index_name):
    print(f"Index '{index_name}' already exists. Loading existing index...")
    # Assuming RAG.from_index is used to load an existing index
    index_path = RAG.from_index(index_name)
else:
    print(f"Index '{index_name}' not found. Creating a new index...")

    # Read documents from all .txt files, excluding _summary.txt files
    documents = read_txt_files_exclude_summary(base_directory)

    # Call the RAG.index method with the list of document contents
    index_path = RAG.index(
        collection=documents,
        index_name=index_name,
        max_document_length=max_document_length,
        split_documents=split_documents
    )

print(f"Index used: {index_path}")

# Example to encode documents without an index if needed
# Assuming the encode function is needed somewhere after the index
encoded_documents = RAG.encode(
    documents=documents,
    bsize=32,
    max_document_length='auto'
)

print(f"Documents encoded.")

Index 'telekom_rag' not found. Creating a new index...
Read document from 'data/MagentaEINS/https_www_telekom_de_hilfe_magenta_eins_vorteil_voraussetzung.txt'
Read document from 'data/MagentaEINS/https_www_telekom_de_hilfe_magenta_eins_prepaid_vorteil_gutaben.txt'
Read document from 'data/MagentaEINS/https_www_telekom_de_hilfe_magenta_eins_vorteil_beschreibung.txt'
Read document from 'data/MagentaEINS/https_www_telekom_de_hilfe_magenta_eins_vorteil_mehrfach.txt'
Read document from 'data/MagentaEINS/https_www_telekom_de_hilfe_magenta_eins_treuebonus_kuendigung.txt'
Read document from 'data/MagentaEINS/https_www_telekom_de_magenta_eins.txt'
Read document from 'data/MagentaEINS/https_www_telekom_de_hilfe_magenta_eins_treuebonus_auszahlung.txt'
Read document from 'data/MagentaEINS/https_www_telekom_de_hilfe_magenta_eins_prepaid_vorteil.txt'
Read document from 'data/MagentaEINS/https_www_telekom_de_hilfe_magenta_eins_vorteil_beauftragen.txt'
Read document from 'data/MagentaEINS/youtube_Die 

  0%|          | 0/50 [00:00<?, ?it/s]/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 29/29 [00:34<00:00,  1.20s/it]

[Sep 19, 16:25:12] [0] 		 avg_doclen_est = 190.8338623046875 	 len(local_sample) = 2,498


[Sep 19, 16:25:12] [0] 		 Creating 8,192 partitions.
[Sep 19, 16:25:12] [0] 		 *Estimated* 476,702 embeddings.
[Sep 19, 16:25:12] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/telekom_rag/plan.json ..


/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/colbert/indexing/collection_indexer.py:256: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sub_sample = torch.load(s

In [ ]:
results = RAG.search(query = question, k=5)
results

In [ ]:
retriever = RAG.as_langchain_retriever(k=5)
#retriever.invoke(question)

In [12]:
# Define the template for generating an answer based on context and a question
telekom_template = """You are an assistant for question-answering tasks for telekom.de help, providing answers to Telekom customers or potential customers. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer or if the provided documents do not contain relevant information, simply say that unfortunately, you cannot assist with this question and please visit www.telekom.de/hilfe for further assistance. 
Use up to four sentences and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt_telekom = ChatPromptTemplate.from_template(telekom_template)

In [13]:
# Function to calculate cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    """
    Computes the cosine similarity between two vectors.
    
    Parameters:
    - vec1 (np.ndarray): The first vector.
    - vec2 (np.ndarray): The second vector.
    
    Returns:
    - float: The cosine similarity between vec1 and vec2.
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2) if (norm_vec1 and norm_vec2) else 0.0

### RAG - Fusion

In [ ]:
# Template for Generating Alternative Questions
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""

# Create a prompt template for generating multiple perspectives of the user's question
prompt_perspectives = ChatPromptTemplate.from_template(template)

# Define a pipeline for generating alternative queries
generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))  # Split the generated output into individual queries
)

# Asynchronous function to print generated queries
async def print_generated_queries(question):
    """
    Generates and prints multiple search queries related to the input question.
    
    Parameters:
    - question (str): The input query for which related search queries are generated.
    """
    queries = generate_queries.invoke({"question": question})
    print("\nGenerated Questions:")
    for q in queries:
        print(f"{q}")

def get_unique_union(documents):
    """
    Returns a unique union of retrieved documents.

    This function takes a list of lists of documents, flattens it, and removes duplicates
    to ensure each document is unique.

    Args:
        documents (list of lists): A list where each element is a list of documents.

    Returns:
        list: A list of unique documents.
    """
    # Flatten the list of lists of documents
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Remove duplicates by converting to a set and then back to a list
    unique_docs = list(set(flattened_docs))
    # Deserialize the documents back into their original form
    return [loads(doc) for doc in unique_docs]

# Define the retrieval chain, which includes generating queries, retrieving documents, and removing duplicates
retrieval_chain = generate_queries | retriever.map() | get_unique_union

# Retrieve multiple documents based on the input question
multi_query_docs = retrieval_chain.invoke({"question": question})

def format_docs(docs, query_embedding):
    """
    Formats the retrieved documents with their source and cosine similarity score.

    This function takes a list of documents and formats them to include the source of each document
    and its cosine similarity to the query embedding.

    Args:
        docs (list): A list of documents retrieved from the database.
        query_embedding (numpy array): The embedding of the user's query.

    Returns:
        str: A formatted string containing the source, similarity score, and content of each document.
    """
    # Initialize a set to track unique sources
    unique_sources = set()
    formatted_docs = []

    for doc in docs:
        # Retrieve the source of the document from its metadata
        source = doc.metadata.get("source")
        # Check if the source is unique
        if source and source not in unique_sources:
            unique_sources.add(source)
            # Compute the embedding of the document's content
            document_embedding = embedding.embed_query(doc.page_content)
            # Calculate cosine similarity between the query and document embeddings
            similarity = cosine_similarity(query_embedding, document_embedding)
            # Use a placeholder message if the document content is empty
            content = doc.page_content.strip() or "This document content is empty."
            # Format the document's source, similarity score, and content
            formatted_docs.append(
                f"Source document: {source}\n\nCosine Similarity: {similarity:.4f}\n\n{content}"
            )

    # Join the formatted documents into a single string
    return "\n\n".join(formatted_docs)

# Define a retrieval and generation (RAG) chain for processing the question and context
rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")} 
    | prompt_telekom
    | model
    | StrOutputParser()
)

async def retrieve_and_format_docs(question):
    """
    Asynchronously retrieves and formats documents for the given question.

    This function retrieves documents relevant to the user's question and formats them with their
    source information and cosine similarity scores.

    Args:
        question (str): The user's question.

    Returns:
        tuple: A tuple containing the answer and formatted documents.
    """
    # Compute the embedding for the user's question
    query_embedding = embedding.embed_query(question)
    # Format the retrieved documents with their cosine similarity scores
    formatted_docs = format_docs(multi_query_docs, query_embedding)
    
    try:
        # Attempt to retrieve an answer using the RAG chain asynchronously
        answer = await rag_chain.invoke({"context": formatted_docs, "question": question})
    except TypeError:
        # Fallback in case of TypeError, invoke the RAG chain synchronously
        answer = rag_chain.invoke({"context": formatted_docs, "question": question})
    
    # Return the answer and the formatted documents
    return answer, formatted_docs

async def main():
    """
    The main asynchronous function to run the complete flow.

    This function handles the process of generating alternative queries, retrieving and formatting
    documents, and printing the final answer along with the source documents.
    """
   
    # Retrieve and format documents, then get the answer
    answer, source_docs = await retrieve_and_format_docs(question)

    # Print the final answer
    print("\nAnswer:", answer)
     # Generate and print alternative queries
    await print_generated_queries(question)
    # Print the source documents used for the answer
    print("\nSources:")
    print(source_docs)

# Execute the main function
await main()